# <span id="0">Casus *Hidden Markov Model* - Deel I</span>

Inhoud:

* **<a href="#1">CpG-islands (CGIs)</a>**

* **<a href="#2">Hidden Markov Models</a>**

* **<a href="#3">CpG-eiland en non-CpG-eiland sequenties</a>**

* **<a href="#4">Toy-data</a>**

* **<a href="#5">Je eigen `HiddenMarkovModel` class</a>**

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import numpy as np

<a id="1" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### CpG-islands (CGIs)

CpG-eilandjes zijn elementen in het genoom die een grote invloed uitoefenen op de regulatie van genexpressie door DNA-methylatie. Hun rol in kankerontwikkeling door abnormale methylatie van tumorsuppressorgenen en oncogenen maakt het een doelwit voor diagnostische en therapeutische benaderingen. Begrip van de dynamiek van CpG-eilandjes biedt inzicht in de moleculaire mechanismen van kanker en mogelijkheden voor innovatieve behandelingsstrategieën.

CpG-eilandjes zijn gebieden in het DNA die een hoge dichtheid hebben aan CpG-dinucleotiden, waarbij "CpG" staat voor een cytosine nucleotide (C) gevolgd door een guanine nucleotide (G) verbonden door een fosfaatgroep (p). Deze eilanden zijn meestal ongewoon rijk aan de CG-combinatie en bevatten vaak geen gemethyleerde cytosines, wat hen onderscheidt van de rest van het genoom waar CpG dinucleotiden vaak gemethyleerd zijn en daardoor minder frequent voorkomen. CpG-eilandjes komen vaak voor in of nabij de promotorregio's van genen, die de expressie van de corresponderende genen reguleren. Ze zijn typisch een paar honderd tot een paar duizend basenparen lang en hebben een GC-gehalte van meer dan 50%, met een CpG-dichtheid hoger dan wat gemiddeld is voor het genoom.

CpG-eilandjes spelen een cruciale rol bij de regulatie van genexpressie via hun invloed op DNA-methylatie, een epigenetisch mechanisme. Dit proces voegt een methylgroep toe aan het cytosine-residu in CpG-dinucleotiden, meestal door de werking van DNA-methyltransferasen. In promotorregio's is methylatie vaak geassocieerd met gen-silencing omdat het de binding van transcriptiefactoren en andere noodzakelijke eiwitten voor genexpressie ontmoedigt. Omgekeerd kan het demethyleren van CpG-eilandjes de expressie van geassocieerde genen activeren, aangezien het de toegankelijkheid van transcriptiefactoren en RNA-polymerase vergroot.

<img src="https://media.licdn.com/dms/image/D4D22AQEdqGnu8ZC80Q/feedshare-shrink_2048_1536/0/1684682580953?e=2147483647&v=beta&t=r6jlPalFOi_YDqJ_tBSPkdZRlvfakvmH1DVf9bnTTEk" width="40%" heigth="40%" />

De epigenetische modificatie van CpG-eilandjes speelt een belangrijke rol in kankerontwikkeling door abnormale genexpressie. In kankercellen worden CpG-eilandjes in de promotorregio's van tumorsuppressorgenen vaak hypergemethyleerd. Dit leidt tot silencing van deze genen, wat bijdraagt aan ongecontroleerde celgroei en proliferatie. Voorbeelden van tumorsuppressorgenen die vaak gehypermethyleerd zijn in kanker omvatten *p16INK4a* en *BRCA1*. Globale hypomethylatie van het genoom kan ook voorkomen in kankercellen, wat resulteert in genomische instabiliteit en de activatie van oncogenen. Dit draagt bij aan tumorvorming en -progressie.

Abnormale methylatiepatronen van CpG-eilandjes kunnen dienen als biomarkers voor de vroege detectie en diagnose van kanker. Epigenetische therapieën, zoals DNA-methylatie-inhibitoren (bijv. azacitidine en decitabine), worden gebruikt om abnormale methylatie om te keren en de normale expressie van tumorsuppressorgenen te herstellen.

Bekijk onderstaande inleidende video en schrijf er een samenvatting van. Zoek zo nodig zelf uitgebreidere achtergrondinformatie over onbekende onderwerpen en vermeld deze bronnen in je samenvatting als referenties. Neem onder andere de bijbehorende [Wikipedia](https://en.wikipedia.org/wiki/CpG_site) pagina door.

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/bc3wtVXyAXo" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

# UITWERKING


Cpg sites zijn dna regio's in het dna waat cytosine wordt gevolgd door een guanine in de 5'-3' richting. Met willekeurige kans zou deze combinatiefrequentie 1/16 keer voor moetn komen echter in gewervelden is dit 1/100. De reden hiervoor is deaminatie. Deze veel voorkomende mutatie van het dna zorgt ervoor dat cytosine, doormiddel van water, wordt omgezet in urasil; Vaker als deze wordt gevold door een Guanine. Dit probleem lost urasil-dna glycosylase efficient op en vormt zelden een probleem. Maar met een gemethyleerde cytosine krijg je een thymine. Dit is ook op te lossen door thymine-dna glycosylase alleen die is erg traag. Waardoor het dna hogere kans krijgt de mutatie te dubliceren en het niet meer is terug te draaien. 

Het is dus onwenselijk om gc rijke regio's te methyleren. Maar de regio's bestaan wel. voornamelijk op gebieden waar amper methylatie plaats vind, met name promoter regio's. Dit zijn zogeheten cpg eilanden. Ze komen voor in de promotor regio's aangezien die niet vaak gemethyleerd worden. 

Bij kanker komt kan het voorkomen dat er methylatie plaatsvind op promotors voor houshoudgenen of oncogenen. Ook is het mogelijk dat er hypermethylatie plaats vind. 



<a id="2" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Hidden Markov Models

CpG-eilandjes kunnen worden gedetecteerd met behulp van verschillende computationele methoden, waarvan een van de meest effectieve methoden het gebruik van Hidden Markov Models (HMMs) is.

Een Hidden Markov Model is een krachtig hulpmiddel voor het modelleren van tijdreeksen en sequentiële data waarin de werkelijke toestanden niet direct zichtbaar zijn, maar waar de uitkomsten afhankelijk zijn van deze toestanden. HMMs maken gebruik van probabilistische berekeningen om te schatten welke verborgen toestanden het meest waarschijnlijk zijn gegeven de waargenomen data, wat ze tot een waardevol instrument maakt in een breed scala aan toepassingen. HMMs worden breed toegepast in verschillende domeinen van de bioinformatica, maar ook bijvoorbeeld in spraakherkenning en natuurlijke taalverwerking of financiële modellering van markttendensen.

Hidden Markov Models zijn zeer geschikt voor de detectie van CpG-eilandjes vanwege hun vermogen om sequenties te modelleren en patronen te herkennen in biologische data. In het kader van CpG-eilandjes vormen de verschillende nucleotiden in het DNA de waargenomen data, en is de aanwezigheid van een CpG-eiland, al dan niet, de verborgen toestand. Door het toepassen van HMMs kunnen wetenschappers efficiënt de locatie van CpG-eilandjes bepalen, wat waardevolle inzichten biedt in genregulatie en kankeronderzoek.

<img src="https://www.researchgate.net/profile/Tomer-Toledo/publication/245563174/figure/fig1/AS:669081116094471@1536532777801/State-transition-diagram-of-a-hidden-Markov-model.png" width="40%" height="40%" />

Een HMM wordt gekarakteriseerd door de volgende eigenschappen:

* **Toestanden / States** Een gegeven aantal verschillende verborgen interne toestanden waarin een geobserveerd systeem zich kan bevinden. In de context van CpG-eilandjesdetectie kunnen de toestanden bijvoorbeeld "in een CpG-eiland" en "buiten een CpG-eiland" zijn.

* **Overgangswaarschijnlijkheden / Transition probabilities** De kans dat het systeem van de ene toestand naar de andere overgaat. Bijvoorbeeld, de kansen om van "buiten een CpG-eiland" naar "in een CpG-eiland" te veranderen, en omgekeerd, of de kansen om binnen dezelfde toestand te blijven.

* **Emissiekansen / Emission probabilities** De kans om een bepaalde uitkomst te observeren vanuit een gegeven toestand. Voor CpG-eilandjes zijn dit de kansen van het voorkomen van verschillende nucleotiden (A, T, C, G) zowel in als buiten de eilandjes.

* **Begintoestandsverdeling / Start probabilities** De initiële waarschijnlijkheden dat het systeem dat zich in een bepaalde toestand bevindt. Een DNA-sequentie kan met zekere kans beginnen met een CpG-eilandje, of niet.

Een HMM maakt, net als elk model, diverse aannamen die de analyse van gegevens vereenvoudigen. De belangrijkste aannamen van een HMM zijn dat (1) het waargenomen systeem zich alléén in een discreet (d.w.z. eindig) aantal verschillende toestanden kan bevinden die variëren in de loop van de reeks, (2) dat de waarnemingen die aan het systeem gedaan worden alléén afhangen van de huidige toestand van het systeem, en (3) dat de kansverdeling omtrent wat de toestand van het systeem op een volgend moment gaat zijn alléén afhangt van wat de toestand van het systeem op een voorgaand moment was.

De toestand van het systeem is zelf niet rechtstreeks waarneembaar; wel waarneembaar zijn de emissies die afhangen van de toestand. De meestgestelde vraag die een HMM kan proberen te beantwoorden is: "gegeven een serie waarnemingen, wat waren de meest waarschijnlijke onderliggende toestanden van het systeem"? Bijvoorbeeld, in ons geval: "gegeven een nucleotidesequentie, welke delen vallen in een CpG-eiland en welke vallen erbuiten"?

Bekijk onderstaande inleidende video en schrijf er een samenvatting van. Zoek zo nodig zelf uitgebreidere achtergrondinformatie over onbekende onderwerpen en vermeld deze bronnen in je samenvatting als referenties. Neem onder andere de bijbehorende [Wikipedia](https://en.wikipedia.org/wiki/Hidden_Markov_model) pagina door.

In [4]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/fX5bYmnHqqE" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In [5]:
# UITWERKING

<a id="3" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### CpG-eiland en non-CpG-eiland sequenties

De onderstaande tabellen geven de experimentele waarschijnlijkheden aan dat een zekere nucleotide (in de rijen) gevolg wordt door een zekere andere nucleotide (in de kolommen) binnen en buiten CpG-eilanden.

Bron: "Biological sequence analysis - Probabilistic models of proteins and nucleic acids", Durbin et al. (1998), hoofdstuk 3, p50-51 ([ref](http://www.mcb111.org/w06/durbin_book.pdf)).

*CGI* (`+`)

|  `+`  |   A   |   C   |   G   |   T   |
| :---: | :---: | :---: | :---: | :---: |
| **A** | 0.180 | 0.274 | 0.426 | 0.120 |
| **C** | 0.171 | 0.368 | 0.274 | 0.188 |
| **T** | 0.161 | 0.339 | 0.375 | 0.125 |
| **G** | 0.079 | 0.355 | 0.384 | 0.182 |

*non-CGI* (`-`)

|  `-`  |   A   |   C   |   G   |   T   |
| :---: | :---: | :---: | :---: | :---: |
| **A** | 0.300 | 0.205 | 0.285 | 0.210 |
| **C** | 0.322 | 0.298 | 0.078 | 0.302 |
| **G** | 0.248 | 0.246 | 0.298 | 0.208 |
| **T** | 0.177 | 0.239 | 0.292 | 0.292 |

Bijvoorbeeld, de kans dat een gegeven C gevolgd wordt door een G is slechts 7.8 % buiten een CpG-eilandje, maar neemt toe tot maar liefst 27.4 % binnen een CpG-eilandje, hetgeen benadrukt hoe sterk CpG-dinucleotiden relatief oververtegenwoordigd zijn binnen CpG-eilandjes. Ga na dat de som van de kansen in elke rij van de tabel moet sommeren tot 100% (op afrondfouten na).

Genereer met python-code een willekeurige sequentie van 300 baseparen lengte overeenkomend met een CpG-eiland en een soortgelijke sequentie van een non-CpG-eiland. Kies telkens het eerste nucleotide willekeurig, en gebruik dan de bovenstaande overgangswaarschijnlijkheden om de sequentie te verlengen. Bepaal van de gegenereerde complete sequenties de kansen op elk nucleotide ($P_A, P_C, P_G, P_T$) evenals de kansen op elk dinucleotide ($P_{AA}, P_{AC}, \ldots, P_{TT}$). Bepaal voor elk dinucleotide ook de ratios $r$ tussen de waargenomen en de verwachte kans, zoals

$$
r_{AC} = \frac{P_{AC}}{P_A \cdot P_C}
$$

Wat kun je verhoudingsgewijs zeggen over hoe vaak C en G afzonderlijk voorkomen, en hoe vaak het CpG-dinucleotide voorkomt? Welke combinaties van nucleotiden zijn het sterkst over- of ondergerepresenteerd?

In [44]:
# UITWERKING
import random

non_cpg = {
    "A": [0.300, 0.205, 0.285, 0.210],
    "C": [0.322, 0.298, 0.078, 0.302],
    "G": [0.248, 0.246, 0.298, 0.208],
    "T": [0.177, 0.239, 0.292, 0.292]
}

cpg_dict = {
    "A": [0.180, 0.274, 0.426, 0.120],
    "C": [0.171, 0.368, 0.274, 0.188],
    "G": [0.079, 0.355, 0.384, 0.182],
    "T": [0.161, 0.339, 0.375, 0.125]
}

nucs = ["A", "C", "G", "T"]

def make_seq(length, matrix):
    sequence = []
    current = random.choice(nucs)
    sequence.append(current)

    for i in range(length - 1):
        prob = matrix[current]
        current = random.choices(nucs, prob)[0]
        sequence.append(current)

    return sequence

def prob_counter(psequence):
    total = {
        "A":0,
        "T":0,
        "C":0,
        "G":0}

    pairs = { 
        "AA":0,
        "AT":0,
        "AC":0,
        "AG":0,
        "TA":0,
        "TT":0,
        "TC":0,
        "TG":0,
        "CA":0,
        "CT":0,
        "CC":0,
        "CG":0,
        "GA":0,
        "GT":0,
        "GC":0,
        "GG":0}
    
    for i in range(len(psequence)):
        total[psequence[i]] += 1
        if i != 0:
            pairs[psequence[i-1] + psequence[i]]+=1

    return divide_dict(total, len(psequence)), divide_dict(pairs, len(psequence))

def divide_dict(my_dict, amount):
    for i in my_dict:
        my_dict[i] /= amount
    return my_dict


cpg_seq = "".join(make_seq(300, non_cpg))

print("CpG island =" , cpg_seq , "\n")

cpg_total, cpg_pairs = prob_counter(cpg_seq) 
print("CpG total probabilaty =", cpg_total , "\n")
print("CpG pair probabilaty =" , cpg_pairs , "\n")

non_cpg_seq = "".join(make_seq(300, non_cpg))

print("Non CpG island =" , non_cpg_seq , "\n")

non_cpg_total, non_cpg_pairs = prob_counter(non_cpg_seq) 
print("Non CpG total probabilaty =", non_cpg_total , "\n")
print("Non CpG pair probabilaty =" , non_cpg_pairs , "\n")


CpG island = GTAGGGCTCTCGCAAGTTGCACTAGCCAACTAGAAATCTTCTTCCTGGTCTTTTTTTGGCTGTCACCCAAGTCGACAGTCCCAAGGGCTCCTTGGCCCCTGTTAAGAGCTGCCAAATTTACTTGACTCCATTTGCTCGCATGTGCCCAGGTGATAAATCCAAGACCACCCTAGAAAGAGGATACCTTGAATTTGCAACAACTCGATTTACCCGAGAATTCCTAGGACACTCGGCAAAGAAGTTCTGCATCACCTTCAGTGGAGGGAAGGGATTTTCTTTGCCACATTAAAAGGAGCCAAG 

CpG total probabilaty = {'A': 0.2633333333333333, 'T': 0.26, 'C': 0.26, 'G': 0.21666666666666667} 

CpG pair probabilaty = {'AA': 0.08666666666666667, 'AT': 0.043333333333333335, 'AC': 0.05333333333333334, 'AG': 0.08, 'TA': 0.03666666666666667, 'TT': 0.1, 'TC': 0.06666666666666667, 'TG': 0.056666666666666664, 'CA': 0.07666666666666666, 'CT': 0.08, 'CC': 0.08333333333333333, 'CG': 0.02, 'GA': 0.06333333333333334, 'GT': 0.03666666666666667, 'GC': 0.056666666666666664, 'GG': 0.056666666666666664} 

Non CpG island = AATCTAGATGCAGGGGAAGAAGATGTACCGGGATGAGTAAGGGAAGCCACAGGAATTACAATTTCTCATGATGACTGACCACTTTCCACACTTTTCACCACCCTATCAAGTAAACATATGGGATGGGGGCTCTACAGGCGGTGCCCTCTTTATTGCATTATTCTCT

<a id="4" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Toy-data

Gegeven zijn drie tafels (overeenkomend met toestanden) die elk een grabbelton bevatten waaruit gekleurde knikkers kunnen worden getrokken (emissies) en met elk een dobbelsteen die bepaalt naar welke volgende tafel je gaat (overgangswaarschijnlijkheden).

| Tafel: |  ❶  |  ❷  |  ❸  |
| -----: | :-: | :-: | :-: |
| **Grabbelton:** | 6x blauw | 2x blauw | 1x blauw |
|                 | 3x geel  | 6x geel  | 0x geel  | 
|                 | 1x groen | 2x groen | 6x groen |
|                 | 2x rood  | 2x rood  | 5x rood  |
| **Dobbelsteen:** | ⚀→① | ⚀→① | ⚀→① |
|                  | ⚁→② | ⚁→② | ⚁→① |
|                  | ⚂→② | ⚂→② | ⚂→① |
|                  | ⚃→② | ⚃→③ | ⚃→① |
|                  | ⚄→③ | ⚄→③ | ⚄→② |
|                  | ⚅→③ | ⚅→③ | ⚅→③ |

Beginnend bij een willekeurige tafel, trek individueel een knikker en noteer de kleur samen met het nummer van de tafel. Gebruik de dobbelsteen om te bepalen naar welke volgende tafel je gaat. Herhaal dit tot je ongeveer een dertigtal waarnemingen hebt gegenereerd en je weer precies terug bent gekomen bij de tafel waar je was begonnen.

Bijvoorbeeld:
| **Beurt:** | 1     | 2     | 3     | 4     | 5     | 6     | 7     | ... | 30    |
| ---------: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :-: | :---: |
| **Tafel:** | ❷     | ❸     | ❶     | ❸     | ❷     | ❷     | ❶     | ... | ❸     |
| **Kleur:** | geel  | groen | blauw | rood  | groen | geel  | blauw | ... | rood  |
| **Worp:**  | ⚃     | ⚀     | ⚄     | ⚄     | ⚂     | ⚀     | ⚂     | ... | ⚄     |

Naast het bepalen van je eigen sequentie, combineer de waarnemingen van al je klasgenoten tot één lange sequentie.

<a id="5" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Je eigen `HiddenMarkovModel` class

Maak een eigen module-bestand genaamd `hmmmodel.py` aan en definieer daarin een klasse `HiddenMarkovModel`. Deze klasse dient een begintoestandsverdeling, overgangswaarschijnlijkheden, en emissiekansen te kunnen bevatten. De klasse dient verder een methode `sample()` te hebben die het mogelijk maakt om een gevraagd aantal waarnemingen te genereren op basis van een gegeven Hidden Markov Model. Voeg implementaties van een aantal standaard dunder-methoden toe (`__init__()`, `__str__()`, `__repr__()`).

Implementeer de functionaliteit van je eigen module zodanig dat deze grotendeels compatibel is met de `CategoricalHMM` klasse van de `hmmlearn` module waarvan je de documentatie [online](https://hmmlearn.readthedocs.io/en/latest/api.html#categoricalhmm) kan vinden. Deze module zit niet in de standaard library van python en dien je zelf wellicht nog te installeren. Merk op dat niet alle functionaliteit uit deze klasse hoeft te worden geïmplementeerd. Vooralsnog gaat het voornamelijk om initialisatie en de `sample()` methode. Later voegen we nog meer methoden aan je klasse toe.

Gebruik je eigen module om sequenties van 1200 toestanden en bijbehorende waarnemingen te genereren voor de eerdere toy-data. De tafelnummers zijn daarin de toestanden en de kleuren knikkers zijn de emissies. Dit zou moeten kunnen met code zoals hieronder.

```python
from hmmmodel import HiddenMarkovModel as HMM
model = HMM(n_components=..., n_features=...)
model.startprob_ = ...
model.transmat_ = ...
model.emissionprob_ = ...
emissions, states = model.sample(1200)
print(model)
```

Toon histogrammen van de toestanden en emissies voor al deze sequenties, en schat *op basis van de gegenereerde sequentie* de overgangswaarschijnlijkheden en emissiekansen. Ga na of deze goed overeenkomen met de instellingen van het Hidden Markov Model.

Vergelijk tenslotte of je eigen klasse soortgelijke resultaten geeft als `hmmlearn.hmm.CategoricalHMM` (d.w.z. met soortgelijke code als hierboven, maar met `from hmmlearn.hmm import CategoricalHMM as HMM`).

In [10]:
# UITWERKING
from hmmmodel import HiddenMarkovModel as HM
model = HM(3, 4)
model.startprob_ = [0.3, 0.4, 0.4]
model.transmat_ = [
    [0.2, 0.2, 0.6],
    [0.5, 0.3, 0.2],
    [0.1, 0.4, 0.5]
]

model.emissionprob_ = [
    [0.7, 0.2, 0.1, 0.0],
    [0.1, 0.4, 0.4, 0.1],
    [0.3, 0.3, 0.2, 0.2]
]

states, emissions = model.sample(1200)
print(model)
print("States\t:", states)
print("Emissions\t:", emissions)

HiddenMarkovModel(n_components=3, n_features=4)
States	: [2, 0, 2, 2, 1, 0, 2, 2, 2, 1, 0, 2, 2, 2, 0, 0, 2, 1, 2, 2, 0, 2, 2, 1, 1, 0, 2, 2, 2, 1, 1, 0, 2, 0, 2, 2, 2, 1, 2, 1, 0, 1, 1, 0, 2, 1, 2, 2, 1, 2, 2, 1, 1, 0, 2, 1, 1, 0, 1, 1, 0, 2, 1, 0, 2, 2, 0, 2, 1, 1, 0, 1, 0, 2, 2, 1, 2, 1, 0, 1, 0, 0, 1, 2, 1, 0, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 0, 2, 2, 0, 2, 1, 0, 2, 2, 1, 2, 2, 1, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 1, 2, 2, 0, 2, 1, 2, 1, 1, 0, 2, 1, 0, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 2, 1, 0, 0, 2, 1, 2, 1, 2, 2, 2, 1, 0, 0, 2, 1, 0, 2, 2, 2, 2, 2, 1, 0, 0, 2, 0, 2, 2, 2, 1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 2, 2, 2, 2, 1, 2, 2, 0, 2, 1, 1, 1, 0, 1, 0, 1, 0, 2, 1, 1, 1, 2, 2, 1, 0, 2, 2, 1, 1, 1, 0, 2, 0, 1, 0, 1, 0, 1, 2, 1, 1, 0, 0, 2, 1, 0, 0, 2, 2, 1, 0, 2, 1, 0, 2, 1, 1, 0, 2, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 0, 1, 0, 2, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 2, 2, 1, 2, 2, 1, 1, 2, 1, 0, 1, 1, 1, 0, 2, 1, 2, 2, 0, 2, 2, 2, 1, 1, 0, 2, 0, 2, 2, 0, 2, 

***

&copy; 2024 - Dave R.M. Langers <d.r.m.langers@pl.hanze.nl>